In [2]:
# Instantiate a small convnet
from keras import models
from keras import layers

model=models.Sequential()
# the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape(height, width, channels)
# 32 or 64 is the number of channels
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [3]:
# feed the last output tensor into a densely-connected classifier network
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

In [4]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images,train_labels),(test_images,test_labels)=mnist.load_data()

train_images=train_images.reshape((60000,28,28,1))
train_images=train_images.astype('float32')/255

test_images=test_images.reshape((10000,28,28,1))
test_images=test_images.astype('float32')/255

train_labels=to_categorical(train_labels)
test_labels=to_categorical(test_labels)

model.compile('rmsprop','categorical_crossentropy',['accuracy'])
model.fit(train_images,train_labels,epochs=5,batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 78s 1ms/step - loss: 0.1736 - acc: 0.9463
Epoch 2/5
60000/60000 [==============================] - 78s 1ms/step - loss: 0.0474 - acc: 0.9854
Epoch 3/5
60000/60000 [==============================] - 76s 1ms/step - loss: 0.0328 - acc: 0.9898
Epoch 4/5
60000/60000 [==============================] - 78s 1ms/step - loss: 0.0246 - acc: 0.9922
Epoch 5/5
60000/60000 [==============================] - 75s 1ms/step - loss: 0.0197 - acc: 0.9937


In [5]:
# evaluate the trained model
test_loss, test_acc=model.evaluate(test_images, test_labels)
print(test_acc)

10000/10000 [==============================] - 4s 380us/step
0.9923


In [6]:
# a convnet without pooling layers
model_no_max_pool=models.Sequential()
model_no_max_pool.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)))
model_no_max_pool.add(layers.Conv2D(64, (3,3), activation='relu'))
model_no_max_pool.add(layers.Conv2D(64, (3,3), activation='relu'))

model_no_max_pool.summary()

# Problems:
# 1. the final layer only contain information coming from 7*7 windows in the initial input.
# 2. the final feature map is too huge to stick to a Dense layer.

# Reasons for downsampling: 
# 1. reduce the number of feature map coefficients to process.
# 2. induce spatial filter hierarchies by making successive convolution layers look at increasingly large windows.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 22, 22, 64)        36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________
